Inisiasi Program

In [1]:
from pf_simulation import PowerFactorySim as pfsim
import os
import pandas as pd

FOLDER_NAME = ''
PROJECT_NAME = '39 Bus New England System'
STUDY_CASE_NAME = '1. Power Flow'
VARIABLES = {
    'name': {
        '*.ElmTerm': ['magnitude','angle'],
        '*.ElmSym': ['rotor_angle', 'out_of_step']
    },
    'symbol': {
        '*.ElmTerm': ['m:u1', 'm:phiu'],
        '*.ElmSym': ['c:firel', 's:outofstep']
    }
}

sim = pfsim(folder_name=FOLDER_NAME, project_name=PROJECT_NAME, study_case_name=STUDY_CASE_NAME)
# sim.app.Show()

# Membuat skenario operasi
SCENARIO = {
    'f_line' : sim.app.GetCalcRelevantObjects('*.ElmLne'),
    'f_location' : [0, 20, 50, 80],
    'f_duration' : [0.1, 0.15, 0.18, 0.2, 0.25],
    'load_level' : [i/100 for i in range(50,115,5)],
}
scenario_metadata = sim.createScenario(scenario_dict=SCENARIO, save=True, flag='y')

# Memuat nilai default load dan gen
file_dir = "output/default_data"
load_file = f"{file_dir}/load_data.parquet"
gen_file = f"{file_dir}/generator_data.parquet"
if all(map(os.path.exists, [load_file, gen_file])):
    load_default, gen_default = (pd.read_parquet(load_file), pd.read_parquet(gen_file))
else:
    load_default, gen_default = sim.loadGenData()
    load_default.to_parquet(load_file, index=False)
    gen_default.to_parquet(gen_file, index=False)

# Mengatur parameter untuk simulasi optimal power flow
sim.opfSetup()

# Mengatur parameter beban ke nilai default
sim.loadSetup(1, load_default)
sim.opfAnalysis('los')
sim.ldfAnalysis()

display(scenario_metadata)

Scenario metadata created


,scenario,load_level,f_line,f_location,f_duration
0,scenario_0001,0.5,Line 01 - 39.ElmLne,0,0.10
1,scenario_0002,0.5,Line 01 - 39.ElmLne,0,0.15
2,scenario_0003,0.5,Line 01 - 39.ElmLne,0,0.18
3,scenario_0004,0.5,Line 01 - 39.ElmLne,0,0.20
4,scenario_0005,0.5,Line 01 - 39.ElmLne,0,0.25
...,...,...,...,...,...
8835,scenario_8836,1.1,Line 01 - 02.ElmLne,80,0.10
8836,scenario_8837,1.1,Line 01 - 02.ElmLne,80,0.15
8837,scenario_8838,1.1,Line 01 - 02.ElmLne,80,0.18
8838,scenario_8839,1.1,Line 01 - 02.ElmLne,80,0.20


Testing OPF

In [5]:
file_dir = "output/generator_dispatch"
for load_lvl_iter in sim.load_level:
    sim.loadSetup(load_lvl_iter, load_default)
    try:
        sim.opfAnalysis('los')
    except AttributeError:
        print(f'{load_lvl_iter:.2f} load, OPF min los failed, trying min dev')
        sim.opfAnalysis('dev')
    
    # Lakukan analisis daya dan ambil dispatch
    gen_dispatch = sim.ldfAnalysis().gen

    # Simpan hasil ke HDF5
    gen_dispatch.to_parquet(f'{file_dir}/scenario_{int(load_lvl_iter*100)}.parquet', index=False)
    print(f'{load_lvl_iter:.2f} load, OPF and LF running successfully')

0.50 load, OPF and LF running successfully
0.55 load, OPF and LF running successfully
0.60 load, OPF and LF running successfully
0.65 load, OPF and LF running successfully
0.70 load, OPF and LF running successfully
0.75 load, OPF min los failed, trying min dev
0.75 load, OPF and LF running successfully
0.80 load, OPF and LF running successfully
0.85 load, OPF and LF running successfully
0.90 load, OPF and LF running successfully
0.95 load, OPF and LF running successfully
1.00 load, OPF and LF running successfully
1.05 load, OPF and LF running successfully
1.10 load, OPF and LF running successfully


,b:tnow_Time,m:u1_Bus 01,m:u1_Bus 02,m:u1_Bus 03,m:u1_Bus 04,m:u1_Bus 05,m:u1_Bus 06,m:u1_Bus 07,m:u1_Bus 08,m:u1_Bus 09,...,s:outofstep_G 01,s:outofstep_G 02,s:outofstep_G 03,s:outofstep_G 04,s:outofstep_G 05,s:outofstep_G 06,s:outofstep_G 07,s:outofstep_G 08,s:outofstep_G 09,s:outofstep_G 10
0,0.01,0.178347,0.762323,0.824419,0.854163,0.864463,0.869713,0.855348,0.850421,0.802233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.02,0.177140,0.757160,0.818776,0.847454,0.856681,0.861817,0.847274,0.842219,0.790992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.03,0.176215,0.753208,0.814584,0.842628,0.851099,0.856166,0.841462,0.836282,0.782171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.04,0.175498,0.750144,0.811428,0.839106,0.847029,0.852054,0.837214,0.831921,0.775190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.05,0.174936,0.747742,0.809021,0.836505,0.844020,0.849017,0.834069,0.828675,0.769622,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,29.96,1.040802,1.025859,1.025120,1.017555,1.020160,1.021647,1.016077,1.015084,1.018357,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2996,29.97,1.040801,1.025858,1.025119,1.017554,1.020159,1.021646,1.016076,1.015083,1.018356,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,29.98,1.040800,1.025857,1.025117,1.017553,1.020158,1.021645,1.016075,1.015082,1.018356,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2998,29.99,1.040799,1.025856,1.025116,1.017552,1.020157,1.021644,1.016074,1.015081,1.018355,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from tqdm import tqdm
import time
import pandas as pd

restart = False
dataset_dir = "D:\Fathin\github\powerfactory_rms\output\dataset" # Harus path lengkap

# Baca file metadata
scenario_list = pd.read_csv("output/scenario_metadata.csv")

# Pastikan kolom 'iter_time' ada
if 'iter_time' not in scenario_list.columns:
    scenario_list['iter_time'] = None

# Filter skenario yang belum dijalankan
if restart:
    scenario_list['iter_time'] = None  # Reset semua iterasi
scenario_list_run = scenario_list[scenario_list['iter_time'].isna()].reset_index(drop=True)

# Total iterasi yang tersisa
total_iterations = scenario_list_run.shape[0]

# Mulai simulasi
with tqdm(total=total_iterations, desc="Simulating", unit="iter") as pbar:
    for _, scen in scenario_list_run.iterrows():
        # Dapatkan parameter scenario
        filename = scen['scenario']
        load_lvl_iter = scen['load_level']
        f_line_iter = scen['f_line']
        f_loc_iter = scen['f_location']
        f_dur_iter = scen['f_duration']

        iter_start_time = time.perf_counter()

        # Simulasi (implementasikan di sini)
        sim.loadSetup(float(load_lvl_iter), load_default)
        try:
            sim.opfAnalysis('los')
        except AttributeError:
            sim.opfAnalysis('dev')
        sim.scSetup(float(f_loc_iter), sim.app.GetCalcRelevantObjects(f_line_iter)[0], float(f_dur_iter))
        sim.rmsSimulation(VARIABLES['symbol'], t_start=-100, t_step=10, t_stop=30)
        sim.getResultRms(dataset_dir, filename)


        iter_end_time = time.perf_counter()
        iter_time = iter_end_time - iter_start_time

        # Update DataFrame dan simpan ke file
        scenario_list.loc[scenario_list['scenario'] == filename, 'iter_time'] = iter_time
        scenario_list.to_csv("output/scenario_metadata.csv", index=False)

        # Update progress bar
        pbar.set_postfix({"current_scenario": filename})
        pbar.update(1)


Simulating: 100%|██████████| 8840/8840 [13:34:23<00:00,  5.53s/iter, current_scenario=scenario_8840]  


In [1]:
import pandas as pd
import os
import glob

# Directory containing Parquet files
parquet_dir = "output/dataset"

# Get all Parquet file paths in the directory
parquet_files = glob.glob(os.path.join(parquet_dir, "*.parquet"))

# List to store DataFrames
dfs = []

# Loop through each file
for file in parquet_files:
    # Read the Parquet file into a DataFrame
    df = pd.read_parquet(file)
    
    # Add a new column with the filename (without extension)
    df['scenario'] = os.path.basename(file)[:-8]
    
    # Append to the list of DataFrames
    dfs.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame back to a single Parquet file
merged_df.to_parquet("output/dataset_IEEE39bus.parquet", index=False)

print("Merged file created with an added 'filename' column!")


Merged file created with an added 'filename' column!


In [4]:
# Untuk query dan load data tiap skenario, tidak disaranakan load semua skenario secara langsung
import duckdb
query = """
    SELECT *
    FROM 'output/dataset_IEEE39bus.parquet'
    WHERE scenario = 'scenario_8840'
"""
dataset = duckdb.query(query).to_df().drop(columns=['scenario'])
dataset


,b:tnow_Time,m:u1_Bus 01,m:u1_Bus 02,m:u1_Bus 03,m:u1_Bus 04,m:u1_Bus 05,m:u1_Bus 06,m:u1_Bus 07,m:u1_Bus 08,m:u1_Bus 09,...,s:outofstep_G 01,s:outofstep_G 02,s:outofstep_G 03,s:outofstep_G 04,s:outofstep_G 05,s:outofstep_G 06,s:outofstep_G 07,s:outofstep_G 08,s:outofstep_G 09,s:outofstep_G 10
0,0.01,0.420507,0.509828,0.603757,0.673974,0.719750,0.729594,0.718666,0.719873,0.845692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.02,0.417583,0.504649,0.600141,0.672366,0.717592,0.727120,0.717090,0.718453,0.841919,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.03,0.415318,0.502756,0.600728,0.675406,0.720303,0.729624,0.720213,0.721594,0.840819,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.04,0.413549,0.502715,0.603502,0.680713,0.725411,0.734595,0.725616,0.726941,0.841206,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.05,0.412159,0.503663,0.607273,0.686908,0.731482,0.740575,0.731902,0.733135,0.842372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,29.96,1.035470,1.041908,1.024731,1.006497,1.011261,1.014920,1.001547,0.999572,1.024997,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2996,29.97,1.035472,1.041902,1.024725,1.006495,1.011261,1.014920,1.001548,0.999574,1.025005,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,29.98,1.035474,1.041896,1.024720,1.006493,1.011261,1.014920,1.001549,0.999576,1.025014,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2998,29.99,1.035476,1.041889,1.024714,1.006491,1.011261,1.014919,1.001550,0.999578,1.025023,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
os.path.basename(file)[:-8]

'scenario_8840'